# Tensor Flow Example

Import and setup the Intelligent Plant clients

In [ ]:
import intelligent_plant.app_store_client as app_store_client
import os
app_store = app_store_client.AppStoreClient(os.environ["ACCESS_TOKEN"])
data_core = app_store.get_data_core_client()

In [ ]:
import intelligent_plant.utility as utility
import matplotlib.pyplot as plt
from pandas import DataFrame

Import Tensorflow and the keras API

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

We can use the data core client to query IP Datasource

Here we get the list of every tag (page size is set to 113 because there are 11 tags in the data source) and use that to query raw for the entire time range there exists data (IP Datasource is on a loop, the dates specified are the original data range), there are 239 points in that range.

In [ ]:
dsn = "IP Datasource 2"
tags = []

page_size = 50
page_num = 1
while True:
    page = data_core.get_tags(dsn, page_num, page_size)
    
    #append the page just fetched into the list of all tags
    tags += page
    
    page_num += 1
    
    #if a page is shorter than the page size requested it is the last page
    if (len(page) < page_size):
        break

#map tages meta data to only be tag name and filter out the "TIME" tag
tag_names = list(filter(lambda x: x != "TIME", map(lambda x: x["Id"], tags)))
all_data = data_core.get_raw_data({dsn: tag_names}, "2014-03-03", "2014-03-03", 239)

We can convert the result data to a data frame

In [ ]:
df = utility.query_result_to_data_frame(all_data)

Remove the timestamp column

In [ ]:
del df['TimeStamp']

Pick a tag as output from the tag list to test against

In [ ]:
df.columns

In [ ]:
tag_to_predict = "LP Flare KO Drum Pump C_Disc_PI"

Take a random sample as training data and the rest can be test data

In [ ]:
train_dataset = df.sample(frac=0.8,random_state=0)
test_dataset = df.drop(train_dataset.index)

Take the tag to predict column and seperate it out

In [ ]:
train_output = train_dataset.pop(tag_to_predict)
test_output = test_dataset.pop(tag_to_predict)

Get some descriptive information about the input data.

We can use the mean and standrad deviation to normalise the data.

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

We can define a model using the Keras API. A sequential model is one where the output of each layer is sequentially fed into the input of the next.

Here I have used a couple of dense neural network layers and the "relu" activation function. The last layer is not "relu" as the output should be between 0 and 1.

This is just an example and may not be the best solution to this problem.

Feel free to make a copy of this notebook and change the sizes, number and types of layers to see how that effects the results.

In [ ]:
model = keras.models.Sequential([
    layers.Dense(512, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
], "DemoNN")

Compiling the network get's it into a state where it's ready to be trained.

Here we select the loss function, the optimzer algorithm and the metrics.

In [ ]:
model.compile(loss='mean_absolute_error',
                optimizer="adam",
                metrics=['mean_absolute_error', 'mean_squared_error'])

Calling `model.summary()` outputs a table showing all the layers in the neural network

In [ ]:
model.summary()

The `model.fit()` function can be used to train the neural network on the provided data.

The `epochs` parameter defines how man iterations through the training set we will use and the `validation_split` parameter determines how much of the training data will be used for validation.

In [ ]:
history = model.fit(normed_train_data, train_output, epochs=20, validation_split = 0.2)

We can use the history object returned from the training step to visualise the training process.

In [ ]:
def plot_history(history):
    hist = DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    #plt.ylim([0,5])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    #plt.ylim([0,20])
    plt.legend()
    plt.show()


plot_history(history)


Each of these graphs is similar but they are showing differnet metrics for the neural netoworks error.

You can see that in both graphs the neural network initally shraply decreases in error and then levels off.

If the validation error (orange) is much greater then the training error (blue) that indicates overfitting. The neural network is now only getting better at instances it's seen before and won't generalise well to unseen data.

The `model.evaluate(..)` function can be used to gernerate metrics about the performance of the neural net on the unseen test data.

This is useful to tell if the neural net is generalising well.

In [ ]:
model.evaluate(normed_test_data, test_output)

The `model.predict(..)` function will run the provided input through the network and return the networks prediction for those input values.

In [ ]:
prediction = model.predict(normed_test_data).flatten()

Putting this prediction into a dataframe allows us to plot the rediction versus the actual value in order to compare them.

In [ ]:
prediction_df = DataFrame({ "actual": test_output, "prediction": prediction })

In [ ]:
prediction_df.plot()

This notebook is adapted from the tensor flow basic regression tutorial

https://www.tensorflow.org/tutorials/keras/basic_regression